In [ ]:
!pip install transformers 
# !pip install nlpaug
!git clone https://github.com/joseph1723/CS376_Final_Project.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 29.5 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 596 kB 71.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'CS376_Final_Project'...
remote: Enumerating objects: 328, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 328 (delta 28), reused 61 (delta 23), pack-reused 254
Receiving objects: 100% (328/328), 139.06 MiB | 15.34 MiB/s, done.
Resolving deltas: 100% (150/150), done.


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

if torch.cuda.is_available() :
  print("CUDA")
  device = torch.device("cuda")
  model.to(device)
else :
  print("CPU")
  device = torch.device("cpu")
  model.to(device)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

CUDA


In [ ]:
class TestDataset(Dataset) :
  #Dataset - English/typo-added/labeled
  def __init__(self, df) :
    self.df = df
  
  def __len__(self) :
    return len(self.df)
  
  def __getitem__(self, idx):
    text = self.df.iloc[idx, 0]
    label = self.df.iloc[idx, 1]
    return text, label

Hyperparameters

In [ ]:
train_rate, test_rate = 0.9, 0.09
itr = 1
p_itr = 100
epochs = 5
batch = 10

In [ ]:
total_df = pd.read_csv('/content/CS376_Final_Project/augmented_data/Dataset_aug_complex_10424_original.csv', sep=',')
#total_df = pd.read_csv('augmented_data/Dataset_aug_complex_10424_original.csv', sep=',')

total_df.dropna(inplace=True)
total_df = total_df[["text", "label"]]
total_df["label"] = [1 if i == "nothate" else 0 for i in total_df["label"]]
print(total_df)
total_dataset = TestDataset(total_df)
total_loader = DataLoader(total_dataset, batch_size=batch, shuffle=True)

                                                    text  label
0      just called this bitch a cum dumpster on the t...      0
1      This dress looks fucking bad on me and I look ...      1
2      If they people don't react well, to you coming...      1
3      "There is no correlation between black populat...      0
4                Fuck yeah, I would kill for some fries.      1
...                                                  ...    ...
10419  would you like to be called brown? Then stop u...      1
10420  i hate being a lesbian living in this country,...      1
10421  The curfew you wanted but now you don't,  make...      1
10422  I love how HE has to come on to this forum, ch...      0
10423  would you like to be called a queer? Then stop...      1

[10424 rows x 2 columns]


In [ ]:
#Train Set
test_df, train_df, _ = np.split(total_df, [int(test_rate*len(total_df)), int(test_rate*len(total_df) + train_rate*len(total_df))])
print(len(test_df), len(train_df))
train_dataset = TestDataset(train_df)
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
test_dataset = TestDataset(test_df)
test_loader = DataLoader(test_dataset, batch_size=batch, shuffle=True)

938 9381


In [ ]:
optimizer = Adam(model.parameters(), lr=1e-6)

total_loss = 0
total_len = 0
total_correct = 0
total_preds = []
X=1

model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        MAX_LEN = max(len(e) for e in encoded_list)
        padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels, return_dict=False)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=X)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()

        total_preds.append(pred)
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0
        itr+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/5] Iteration 100 -> Train Loss: 0.7039, Accuracy: 0.522
[Epoch 1/5] Iteration 200 -> Train Loss: 0.6804, Accuracy: 0.569
[Epoch 1/5] Iteration 300 -> Train Loss: 0.6801, Accuracy: 0.554
[Epoch 1/5] Iteration 400 -> Train Loss: 0.6689, Accuracy: 0.593
[Epoch 1/5] Iteration 500 -> Train Loss: 0.6733, Accuracy: 0.595
[Epoch 1/5] Iteration 600 -> Train Loss: 0.6709, Accuracy: 0.592
[Epoch 1/5] Iteration 700 -> Train Loss: 0.6517, Accuracy: 0.597
[Epoch 1/5] Iteration 800 -> Train Loss: 0.6646, Accuracy: 0.595
[Epoch 1/5] Iteration 900 -> Train Loss: 0.6333, Accuracy: 0.650
[Epoch 2/5] Iteration 1000 -> Train Loss: 0.6461, Accuracy: 0.607
[Epoch 2/5] Iteration 1100 -> Train Loss: 0.6061, Accuracy: 0.672
[Epoch 2/5] Iteration 1200 -> Train Loss: 0.6161, Accuracy: 0.637
[Epoch 2/5] Iteration 1300 -> Train Loss: 0.6056, Accuracy: 0.656
[Epoch 2/5] Iteration 1400 -> Train Loss: 0.5879, Accuracy: 0.678
[Epoch 2/5] Iteration 1500 -> Train Loss: 0.5682, Accuracy: 0.698
[Epoch 2/5] Iterati

In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 33.9 MB/s 


In [ ]:
Test_uniform_df = pd.read_csv("/content/CS376_Final_Project/datasets/2020-12-31-DynamicallyGeneratedHateDataset-entries-v0.2.csv",sep=',')
Test_uniform_df.dropna(inplace=True)
Test_uniform_df = Test_uniform_df[['text','label']]
Test_uniform_df_hate = Test_uniform_df.loc[Test_uniform_df['label'] == 'hate',:]
Test_uniform_df_nothate = Test_uniform_df.loc[Test_uniform_df['label'] == 'nothate',:]
Test_uniform_df = pd.concat([Test_uniform_df_hate[:200], Test_uniform_df_nothate[:200]], axis=0).reset_index(drop=True)
#Test_uniform_df = Test_uniform_df.sample(frac=1).reset_index(drop=True)
Test_uniform_df["label"] = [1 if i == "nothate" else 0 for i in Test_uniform_df["label"]]

test_uniform_dataset = TestDataset(Test_uniform_df)
test_uniform_loader = DataLoader(test_uniform_dataset, batch_size=batch, shuffle=True)

In [ ]:
from torchmetrics import F1Score

model.eval()

total_loss = 0
total_len = 0
total_correct = 0
total_preds = []

for text, label in test_uniform_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    MAX_LEN = max(len(e) for e in encoded_list)
    padded_list =  [e + [0] * (MAX_LEN-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels, return_dict=False)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    total_preds.append(pred)

    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

labels = torch.tensor(Test_uniform_df['label']).to(device)
total_preds = torch.cat(total_preds).to(device)
f1 = F1Score(num_classes=2).to(device)
F1_Score = f1(total_preds, labels)
print('F1 Score : ', F1_Score)

print('Test accuracy: ', total_correct / total_len)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


F1 Score :  tensor(0.5125, device='cuda:0')
Test accuracy:  0.7775


# Accuracy (epoch=5) : 0.7775
# F1-Score : 0.5125